# Visualization of flash response
This notebook allow to explore flash response and its features.
Data must be created using [PRO_get_features_of_flash_response](PRO_get_features_of_flash_response.ipynb)

In [1]:
from configparser import ConfigParser, ExtendedInterpolation

import h5py
from ipywidgets import IntSlider, interact, Dropdown
import numpy as np
import matplotlib.pyplot as plt

# %matplotlib notebook 
config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../../config.ini')

['../../config.ini']

In [2]:
exp_name = config['EXP']['name']
sync_file = config['FILES']['sync']
source_file = config['FILES']['processed']
suffix= '_' + config['PROCESSED']['suffix']
print('SOURCE FILE: ' + source_file)

protocol = 'flash'
intensities = [50, 100, 150, 200, 255]
nd_filters = ['nd3',]
extra = ['cyan']

SOURCE FILE: /home/vnc/Documents/Alzheimer/Degus/MR-0061/data/processed_protocols/MR-0061_oldcircus_analysis_of_protocols.hdf5


In [3]:
panalysis = h5py.File(source_file, 'r')

prefix = 'nd3'
type_group = '/flash/{}/255_cyan/type/'.format(prefix)

no_valid_units = []
valid_units = []
all_valid_units = []
for key in panalysis[type_group]:
    all_valid_units.append(key)
    if panalysis[type_group+key][...]:
        valid_units.append(key)
    else:
        no_valid_units.append(key)
print('Numbers of valid units: {:d}\nNumber of not valid unit: {:d}'.format(len(valid_units),len(no_valid_units)))

Numbers of valid units: 246
Number of not valid unit: 104


# Visualization flash response for one ND={2,3,4,5} and all intensities together

In [4]:
def plot_fit(protocol, nd, extra, key):
    fields = dict(protocol=protocol, nd=nd, extra=extra)
    base_group = '/{protocol}/{nd}/{intensity:03d}_{extra}/'
    group = base_group + 'est_psth/'
    dset = base_group + 'est_psth/{template}'
    
    plt.figure(figsize=(10,6))
    for kintensity in intensities:
        fields['intensity'] = kintensity
        fields['template'] = key
        time = panalysis[group.format(**fields)].attrs['time']
        response = panalysis[dset.format(**fields)][...]
        s_on, f_on, s_off, f_off = panalysis[group.format(**fields)].attrs['bounds']
        plt.plot(time, response, label=str(kintensity), alpha=0.5)
        max_amp = response.max()
        plt.vlines(s_off,0,max_amp)
    plt.xlabel('time [s]')
    plt.ylabel('instantaneous firing rate [spk/bins]')
    plt.title('Flash response for {} in different intensities. Filter {}'.format(key, prefix.upper()))
    plt.legend()
    plt.show()
    plt.close()

In [5]:
# interact(plot_fit, kidx=IntSlider(min=0, max=len(temporal_sta)-1, step=1, value=172));
interact(plot_fit,
         protocol=protocol,
         nd=Dropdown(options=nd_filters),
         extra=Dropdown(options=extra),
         key=Dropdown(options=valid_units));

interactive(children=(Text(value='flash', description='protocol'), Dropdown(description='nd', options=('nd3',)…

# Visualization flash response for ND(2,3,4,5) and all intensities together

In [6]:
def plot_nd(protocol, extra, key):
    nd = nd_filters
    fields = dict(protocol=protocol, extra=extra, template=key)
    base_group = '/{protocol}/{nd}/{intensity:03d}_{extra}/'
    group = base_group + 'est_psth/'
    dset = base_group + 'est_psth/{template}'
    fig, ax = plt.subplots(len(nd),1,figsize=(10,15), sharex=True, sharey=True)
    ax = ax.flatten() if len(nd) > 1 else [ax]
    for kfilter, kax in zip(nd, ax):
        fields['nd'] = kfilter
        for kintensity in intensities:
            fields['intensity'] = kintensity
            print(group.format(**fields))
            time = panalysis[group.format(**fields)].attrs['time']
            response = panalysis[dset.format(**fields)][...]
            s_on, f_on, s_off, f_off = panalysis[group.format(**fields)].attrs['bounds']
            kax.plot(time, response, label=str(kintensity), alpha=0.5)
            max_amp = response.max()
            kax.vlines(s_off,0,max_amp)
        kax.set(xlabel='time [s]', ylabel='instantaneous firing rate [spk/bins]',
               title='Filter {}'.format(kfilter.upper()))
        kax.legend()
        kax.grid(b=True, which='both', ls=':', alpha=0.3)
#         kax.grid(which='major', ls=':', alpha=0.3)
    fig.suptitle('Flash response for {} in different intensities.'.format(key))
#     plt.tight_layout()
    plt.show()
    plt.close()

In [7]:
# interact(plot_fit, kidx=IntSlider(min=0, max=len(temporal_sta)-1, step=1, value=172));
interact(plot_nd,
         protocol=protocol,
         extra=Dropdown(options=extra),
         key=Dropdown(options=all_valid_units));

interactive(children=(Text(value='flash', description='protocol'), Dropdown(description='extra', options=('cya…

# Visualization all intensities for an ND(2,3,4,5)

In [14]:
def plot_intensities(protocol, nd, extra, key):
    fields = dict(protocol=protocol, extra=extra, template=key, nd=nd)
    base_group = '/{protocol}/{nd}/{intensity:03d}_{extra}/'
    group = base_group + 'est_psth/'
    dset = base_group + 'est_psth/{template}'
    flash_type = base_group + 'type/{template}'
    flash_char = base_group + 'char/{template}'
    
    fig, ax = plt.subplots(3, 3, figsize=(15,12), sharex=True, sharey=True)
    fig.suptitle('Flash response for {} in different intensities. Filter {}'.format(key, nd.upper()))
    for kintensity, kax in zip(intensities, ax.flatten()):
        fields['intensity'] = kintensity
        time = panalysis[group.format(**fields)].attrs['time']
        response = panalysis[dset.format(**fields)][...]
        s_on, f_on, s_off, f_off = panalysis[group.format(**fields)].attrs['bounds']
        chars = panalysis[flash_char.format(**fields)][...]
        kax.plot(time, response, label=str(kintensity), alpha=0.5)
        max_amp = response.max()
        kax.vlines(s_off,0,max_amp)
        kax.vlines(chars[0]+s_on, 0, max_amp, alpha=0.5, color='r', linestyle='--')
        kax.vlines(chars[1]+s_off, 0, max_amp, alpha=0.5, color='b', linestyle='--')
        flash_t = panalysis[flash_type.format(**fields)][...]
        kax.set(xlabel='time [s]', ylabel='inst fr [spk/bins]', 
               title='Type {}'.format(kintensity),
               )
        kax.legend()
    plt.show()
    plt.close()

In [15]:
# interact(plot_fit, kidx=IntSlider(min=0, max=len(temporal_sta)-1, step=1, value=172));
interact(plot_intensities, 
         protocol=protocol,
         nd=nd_filters,
         extra=extra,
         key=Dropdown(options=valid_units));

interactive(children=(Text(value='flash', description='protocol'), Dropdown(description='nd', options=('nd3',)…

# Close HDF5 file

In [8]:
panalysis.close()